In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import bz2
import json
import os
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pandas==1.3.4

     |████████████████████████████████| 11.3 MB 4.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


In [3]:
pd.__version__

'1.3.4'

# Utility function and paths variables

In [4]:
def apply_to_stream(f, input_file, chunksize=1_000_000):
    with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in tqdm(df_reader):
            f(chunk)


def apply_to_stream_and_save(f, input_file, output_file, chunksize=1_000_000):
    with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        with bz2.open(output_file, 'wb') as out_file:
            for df_chunk in tqdm(df_reader):
                write_df_chunk_to_file(f(df_chunk), out_file)


def apply_to_all_files_stream_and_save(f, all_files, output_file, chunksize=1_000_000):
    # Open only once the output file, so just append all the content to it
    with bz2.open(output_file, 'wb') as out_file:
        for input_file in all_files:
            with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
                    for df_chunk in tqdm(df_reader):
                        write_df_chunk_to_file(f(df_chunk), out_file)
                        
            print(f'==> File "{input_file}" processed')


def write_df_chunk_to_file(df_chunk, out_file):
    out_file.write(df_chunk.to_json(orient='records', lines=True).encode('utf-8'))

In [5]:
data_out_folder = '/content/drive/Shareddrives/ada-teamphilippe/data/'

data_folder = '/content/drive/Shareddrives/ada-teamphilippe/Quotebank/'
quotes_2015 = data_folder + 'quotes-2015.json.bz2'
quotes_2016 = data_folder + 'quotes-2016.json.bz2'
quotes_2017 = data_folder + 'quotes-2017.json.bz2'
quotes_2018 = data_folder + 'quotes-2018.json.bz2'
quotes_2019 = data_folder + 'quotes-2019.json.bz2'
quotes_2020 = data_folder + 'quotes-2020.json.bz2'
all_files = [quotes_2015, quotes_2016, quotes_2017, quotes_2018, quotes_2019, quotes_2020]

wikidata_parquet = data_folder + 'speaker_attributes.parquet'

# Formats of the differents features

In [ ]:
def get(nb):
    with pd.read_json(quotes_2015, lines=True, compression='bz2', chunksize=nb) as df_reader:
        for chunk in df_reader:
            return chunk

In [ ]:
first_chunk = get(50)

In [ ]:
first_chunk.head(50)

In [ ]:
first_chunk.iloc[1]['speaker'] == 'None'

We will replace these string 'None' by proper Nan to ease the processing with pandas.

In [ ]:
first_chunk['quoteID'].describe()

In [ ]:
first_chunk['quotation'].describe()

In [ ]:
first_chunk['speaker'].describe()

In [ ]:
first_chunk['qids'].describe()

In [ ]:
first_chunk['date'].describe()

We will need to transform the date column type to a proper datetime type.

In [ ]:
first_chunk['numOccurrences'].describe()

In [ ]:
first_chunk['numOccurrences'].hist()
plt.show()

In [ ]:
first_chunk['probas'].describe()

The probas feature already contains python list.

In [ ]:
first_chunk['urls'].describe()

In [ ]:
first_chunk['phase'].describe()

It seems that the column 'phase' does contain much information since it is always the same value (to be confirmed on the entire dataset). If it is the case, we can just drop it.

#### Total number of records

In [6]:
def count_nb_rows_in_file(input_file):
  count = 0

  with pd.read_json(input_file, lines=True, compression='bz2', chunksize=750_000) as df_reader:
      for chunk in df_reader:
          count += chunk.shape[0]

  print(f'Total number of records : {count}')

In [ ]:
count_nb_rows_in_file(quotes_2015)

In [ ]:
count_nb_rows_in_file(quotes_2016)

In [ ]:
count_nb_rows_in_file(quotes_2017)

In [ ]:
count_nb_rows_in_file(quotes_2018)

In [ ]:
count_nb_rows_in_file(quotes_2019)

In [ ]:
count_nb_rows_in_file(quotes_2020)

# Cleaning
As our project idea is based not only on the quote but also on the author, we decided to drop the rows that were having None as speaker or the rows where the speaker has a probability less than 0.5.

In [7]:
def cast_date_remove_authors_low_proba(df_chunk):
    def create_col_author_highest_proba(row):
        max_proba = -1.0
        max_author = None
        
        for author, proba in row['probas']:
            if float(proba) > max_proba:
                max_proba = float(proba)
                max_author = author
                
        return max_author, max_proba
        
        
    # Cast the date column to datetime
    df_chunk['date'] = pd.to_datetime(df_chunk['date'])
    
    # Cast the string 'None' for the speaker column to proper np.nan
    df_chunk['speaker'] = df_chunk['speaker'].replace('None', np.nan)
    
    # Drop all the rows where the author is nan
    df_chunk.dropna(subset=['speaker'], inplace=True)
    
    tmp = pd.DataFrame()
    # Create 2 new columns with author, proba that has the highest proba
    tmp[['author_highest_proba', 'highest_proba']] = df_chunk.apply(create_col_author_highest_proba, axis=1, result_type='expand')
    
    # Check if for some rows the author is not the author with the highest proba
    if not df_chunk['speaker'].equals(tmp['author_highest_proba']):
        print('========================================================================')
        print('The column "speaker" is not equal to the column "author_highest_proba" !')
        print('========================================================================')
        
        # Print where the 2 columns are different
        print(df_chunk[np.argwhere(not df_chunk['speaker'].equals(tmp['author_highest_proba']))])
        
    # Drop the rows where the highest proba is < 0.5
    mask = tmp['highest_proba'] < 0.5
    nb_rows_dropped = mask.sum()
    df_chunk = df_chunk[~mask]
    
    return df_chunk, nb_rows_dropped

In [ ]:
def rename_rec(i):
    try:
        os.rename(output_file, f'data/dataset_old_{i}.json.bz2')
    except OSError:
        rename_rec(i+1)

# Output file
output_file = data_out_folder + 'cleaned_dataset_all_years.json.bz2'

# Rename old file if it already exists
if os.path.isfile(output_file):
    # File already exists, rename it
    rename_rec(0)

#apply_to_all_files_stream_and_save(cast_date_remove_authors_low_proba, all_files, output_file)

28it [1:10:40, 151.45s/it]


==> File "data/quotes-2015.json.bz2" processed


19it [36:17, 114.59s/it]


==> File "data/quotes-2016.json.bz2" processed


36it [1:17:53, 129.81s/it]


==> File "data/quotes-2017.json.bz2" processed


11it [40:56, 246.86s/it]

In [8]:
def clean_and_sample_file_and_save(f, input_file, output_file, chunksize=750_000):
  total_nb_rows = 0
  with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
      with bz2.open(output_file, 'wb') as out_file:
          for df_chunk in tqdm(df_reader):
            df_result, nb_rows_dropped = f(df_chunk)
            
            # Write result chunk to file
            write_df_chunk_to_file(df_result, out_file)
            total_nb_rows += nb_rows_dropped

  print(f'Number of rows dropped in file {input_file} : {total_nb_rows}')

In [9]:
cleaned_quotes_2015 = data_out_folder + 'cleaned_quotes_2015.json.bz2'
cleaned_quotes_2016 = data_out_folder + 'cleaned_quotes_2016.json.bz2'
cleaned_quotes_2017 = data_out_folder + 'cleaned_quotes_2017.json.bz2'
cleaned_quotes_2018 = data_out_folder + 'cleaned_quotes_2018.json.bz2'
cleaned_quotes_2019 = data_out_folder + 'cleaned_quotes_2019.json.bz2'
cleaned_quotes_2020 = data_out_folder + 'cleaned_quotes_2020.json.bz2'

In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2015, cleaned_quotes_2015)

28it [1:02:01, 132.91s/it]


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2015.json.bz2 : 662490


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2015.json.bz2 : 662490

In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2016, cleaned_quotes_2016)

19it [44:34, 140.78s/it]


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2016.json.bz2 : 453279


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2016.json.bz2 : 453279


In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2017, cleaned_quotes_2017)

36it [1:20:11, 133.64s/it]


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2017.json.bz2 : 859026


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2017.json.bz2 : 859026

In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2018, cleaned_quotes_2018)

37it [1:27:24, 141.75s/it]


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2018.json.bz2 : 883951


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2018.json.bz2 : 883951

In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2019, cleaned_quotes_2019)

30it [1:07:49, 135.64s/it]

Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2019.json.bz2 : 695779


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2019.json.bz2 : 695779


In [ ]:
clean_and_sample_file_and_save(cast_date_remove_authors_low_proba, quotes_2020, cleaned_quotes_2020)

7it [13:39, 117.01s/it]


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2020.json.bz2 : 160320


Number of rows dropped in file /content/drive/Shareddrives/ada-teamphilippe/Quotebank/quotes-2020.json.bz2 : 160320

# Sampling the dataset
Since the original dataset is very large, it is quite cumbersome to do our first analyses on it because it takes a lot of time. Therefore to get a first glance at the data, test our functions, etc, we sample the original dataset to obtain a much smaller one.


Now arises the question on how should we sample it ? Should we just sample uniformly at random a fixed number of quotes per year ? Should we look for biases in the data as seen in the "Observation studies" lecture and sample the dataset accordingly ? For example, we could look at the proportion of women/men authors in each year and decide to balance the samples according to this. However, this would require to merge the dataset with the Wikidata parquet file and so on. So for now, we decided to start by sampling uniformly at random 500'000 quotes from each year from the cleaned dataset we created above. Since the sampling is random, this should conserve the biases of the origininal dataset and therefore we will be able to detect them faster (in terms of computational time). Obviously, we will run our analyses on the full dataset once in the end to get more representative results.

In [ ]:
count_nb_rows_in_file(cleaned_quotes_2015)

Total number of records : 13195548


Total number of records in file "cleaned_quotes_2015" : 13195548

In [ ]:
count_nb_rows_in_file(cleaned_quotes_2016)

Total number of records : 8653435


Total number of records in file "cleaned_quotes_2016" : 8653435

In [ ]:
count_nb_rows_in_file(cleaned_quotes_2017)

Total number of records : 16474272


Total number of records in file "cleaned_quotes_2017" : 16474272

In [ ]:
count_nb_rows_in_file(cleaned_quotes_2018)

Total number of records : 16786468


Total number of records in file "cleaned_quotes_2018" : 16786468


In [ ]:
count_nb_rows_in_file(cleaned_quotes_2019)

Total number of records : 13487515


Total number of records in file "cleaned_quotes_2019" : 13487515

In [ ]:
count_nb_rows_in_file(cleaned_quotes_2020)

Total number of records : 3283285


Total number of records in file "cleaned_quotes_2020" : 3283285

In [10]:
def sample_dataset(input_file, output_file, nb_samples, seed=1, chunksize=750_000):
  # First need to get back the quoteID of all the rows in the file
  quoteIDs = []
  with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
      for df_chunk in tqdm(df_reader):
        quoteIDs += df_chunk['quoteID'].tolist()

  print('==> quoteIDs processed')

  # Choose nb_samples uniformly at random
  rng = np.random.default_rng(seed=seed)
  quoteIDs_samples = rng.choice(quoteIDs, nb_samples, replace=False)

  with pd.read_json(input_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
      with bz2.open(output_file, 'wb') as out_file:
          for df_chunk in tqdm(df_reader):
            # Keep only chosen rows
            df_result = df_chunk.loc[df_chunk['quoteID'].isin(quoteIDs_samples), :]
            
            # Write result chunk to file
            write_df_chunk_to_file(df_result, out_file)

  print(f'==> Succesfully sampled {nb_samples} out of {len(quoteIDs)} from file "{input_file}"')

In [11]:
nb_samples = 300_000
cleaned_quotes_2015_sampled = data_out_folder + 'cleaned_quotes_2015_sampled.json.bz2'
cleaned_quotes_2016_sampled = data_out_folder + 'cleaned_quotes_2016_sampled.json.bz2'
cleaned_quotes_2017_sampled = data_out_folder + 'cleaned_quotes_2017_sampled.json.bz2'
cleaned_quotes_2018_sampled = data_out_folder + 'cleaned_quotes_2018_sampled.json.bz2'
cleaned_quotes_2019_sampled = data_out_folder + 'cleaned_quotes_2019_sampled.json.bz2'
cleaned_quotes_2020_sampled = data_out_folder + 'cleaned_quotes_2020_sampled.json.bz2' 

In [12]:
sample_dataset(cleaned_quotes_2015, cleaned_quotes_2015_sampled, nb_samples)

18it [22:48, 76.06s/it]


==> quoteIDs processed


18it [24:13, 80.72s/it]


==> Succesfully sampled 300000 out of 13195548 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2015.json.bz2"


In [13]:
sample_dataset(cleaned_quotes_2016, cleaned_quotes_2016_sampled, nb_samples)

12it [15:59, 79.95s/it]


==> quoteIDs processed


12it [17:32, 87.67s/it]


==> Succesfully sampled 300000 out of 8653435 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2016.json.bz2"


In [14]:
sample_dataset(cleaned_quotes_2017, cleaned_quotes_2017_sampled, nb_samples)

22it [33:38, 91.74s/it]


==> quoteIDs processed


22it [37:55, 103.42s/it]


==> Succesfully sampled 300000 out of 16474272 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2017.json.bz2"


In [15]:
sample_dataset(cleaned_quotes_2018, cleaned_quotes_2018_sampled, nb_samples)

23it [31:09, 81.29s/it]


==> quoteIDs processed


23it [34:28, 89.95s/it]

==> Succesfully sampled 300000 out of 16786468 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2018.json.bz2"


In [16]:
sample_dataset(cleaned_quotes_2019, cleaned_quotes_2019_sampled, nb_samples)

18it [24:23, 81.29s/it]


==> quoteIDs processed


18it [25:47, 85.97s/it]


==> Succesfully sampled 300000 out of 13487515 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2019.json.bz2"


In [17]:
sample_dataset(cleaned_quotes_2020, cleaned_quotes_2020_sampled, nb_samples)

5it [04:38, 55.72s/it]


==> quoteIDs processed


5it [06:04, 72.91s/it]


==> Succesfully sampled 300000 out of 3283285 from file "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2020.json.bz2"


To ease the processing on the sampled dataset, we will merge the 6 sampled files into a single one.

In [19]:
# Now let's merge these 6 sampled files
apply_to_all_files_stream_and_save(lambda x: x,
                                   [cleaned_quotes_2015_sampled, cleaned_quotes_2016_sampled, cleaned_quotes_2017_sampled,
                                    cleaned_quotes_2018_sampled, cleaned_quotes_2019_sampled, cleaned_quotes_2020_sampled],
                                   data_out_folder + 'cleaned_quotes_sampled_all.json.bz2'
                                   )

1it [00:55, 55.43s/it]


==> File "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2015_sampled.json.bz2" processed


1it [00:58, 58.44s/it]


==> File "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2016_sampled.json.bz2" processed


1it [01:02, 62.86s/it]


...

==> File "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2018_sampled.json.bz2" processed


1it [01:00, 60.31s/it]


==> File "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2019_sampled.json.bz2" processed


1it [01:03, 63.82s/it]


==> File "/content/drive/Shareddrives/ada-teamphilippe/data/cleaned_quotes_2020_sampled.json.bz2" processed
